# Summarization of Financial Data Using Hugging Face NLP models

This notebook shows model developers how to document a natural language processing (NLP) model using the ValidMind Developer Framework. The use case is a summarization of financial news based on a dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. It shows you how to set up the ValidMind Developer Framework, initialize the client library, and load the dataset, followed by running the model validation tests provided by the framework to quickly generate documentation about the data and model.

## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on documentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, we recommend going through the following resources first:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

::: {.callout-tip}
### New to ValidMind? 
For access to all features available in this notebook, create a free ValidMind account. 

Signing up is FREE — [**Sign up now**](https://app.prod.validmind.ai)
:::


If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).

## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:

In [ ]:
%pip install -q validmind

## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

Get your code snippet by creating a documentation project:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to Go to **Documentation Projects** and click **Create new project**.

3. Select **`[Demo] Hugging Face - Text Summarization`** and **`Initial Validation`** for the model name and type, give the project a unique  name to make it yours, and then click **Create project**.

4. Go to **Documentation Projects** > **YOUR_UNIQUE_PROJECT_NAME** > **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace the code below with the code snippet from your project ##


import validmind as vm

vm.init(
  api_host = "https://api.prod.validmind.ai/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)


### Preview the documentation template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

### Helper functions

Let's define the following functions to help visualize datasets with long text fields:

In [ ]:
import textwrap

from IPython.display import display, HTML
from tabulate import tabulate

def _format_cell_text(text, width=50):
    """Private function to format a cell's text."""
    return '\n'.join([textwrap.fill(line, width=width) for line in text.split('\n')])

def _format_dataframe_for_tabulate(df):
    """Private function to format the entire DataFrame for tabulation."""
    df_out = df.copy()

    # Format all string columns
    for column in df_out.columns:
        if df_out[column].dtype == object:  # Check if column is of type object (likely strings)
            df_out[column] = df_out[column].apply(_format_cell_text)
    return df_out

def _dataframe_to_html_table(df):
    """Private function to convert a DataFrame to an HTML table."""
    headers = df.columns.tolist()
    table_data = df.values.tolist()
    return tabulate(table_data, headers=headers, tablefmt="html")

def display_formatted_dataframe(df, num_rows=None):
    """Primary function to format and display a DataFrame."""
    if num_rows is not None:
        df = df.head(num_rows)
    formatted_df = _format_dataframe_for_tabulate(df)
    html_table = _dataframe_to_html_table(formatted_df)
    display(HTML(html_table))

### Load the dataset

The CNN Dailymail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail (https://huggingface.co/datasets/cnn_dailymail). The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')
train_df = cnn_dataset.data['train'].to_pandas()
val_df = cnn_dataset.data['validation'].to_pandas()
test_df = cnn_dataset.data['test'].to_pandas()
train_df = train_df[['article','highlights']]
train_df = train_df.head(20)

display_formatted_dataframe(train_df, num_rows=5)

In [ ]:
df = train_df.head(100)
# Load a test dataset with 100 rows only
vm_ds = vm.init_dataset(
    dataset=df,
    text_column="article",
    target_column="highlights",
)

## NLP data quality tests

Before we proceed with the analysis, it's crucial to ensure the quality of our NLP data. We can run the `data_preparation` section of the template to validate the data's integrity and suitability:

In [ ]:
text_data_test_plan = vm.run_documentation_tests(
    section="data_preparation",
    inputs = {"dataset":vm_ds}
)

In [ ]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

summarizer_model = pipeline(
    task="summarization",
    model=model,
    tokenizer = tokenizer,
    min_length=0,
    max_length=60,
    truncation=True,
    model_kwargs={"cache_dir": '/Documents/Huggin_Face/'},
)  # Note: We specify cache_dir to use predownloaded models.

In [ ]:
df_test = df.head(10)

vm_test_ds = vm.init_dataset(
    dataset=train_df,
    text_column="article",
    target_column="highlights",
)

In [ ]:
vm_model = vm.init_model(
    summarizer_model,
    test_ds=vm_test_ds,
)

### Run model validation tests

It's possible to run a subset of tests on the documentation template by passing a `section` parameter to `run_documentation_tests()`. Let's run only the tests that evaluate the model's overall performance, including summarization metrics, by selecting the `model_development` section of the template:

In [ ]:
config={
    "rouge_metric": {
        "rouge_metrics": ["rouge-1","rouge-2", "rouge-l"],
    },
}
summarization_results = vm.run_documentation_tests(
    section="model_development",
    inputs = {"model":vm_model},
    config=config,
)

## Next steps

You can look at the results of this test suite right in the notebook where you ran the code, as you would expect. But there is a better way: view the prompt validation test results as part of your model documentation right in the ValidMind Platform UI: 

1. Log back into the [Platform UI](https://app.prod.validmind.ai) 

2. Go to **Documentation Projects** > **YOUR_DOCUMENTATION_PROJECT** > **Documentation**.

3. Expand **2. Data Preparation** or **3. Model Development** to review all test results.

What you can see now is a more easily consumable version of the prompt validation testing you just performed, along with other parts of your documentation project that still need to be completed. 

If you want to learn more about where you are in the model documentation process, take a look at [How do I use the framework?](https://docs.validmind.ai/guide/get-started-developer-framework.html#how-do-i-use-the-framework).

